In [17]:
#Functions used throughout the notebook

def bin_variables(df, columns_df, percent = 0.05):
  for col in columns_df:
    if not pd.api.types.is_numeric_dtype(df[col]):
      for value, count in df[col].value_counts().iteritems():
        if count/ len(df) < percent:
          df.loc[df[col] == value, col] = 'Other'

  return df

def handle_missing(df, percent = 0.5):
  for col in df:
    if df[col].isna().sum()/len(df) > percent:
      df.drop(columns = [col], inplace = True)

      print('Dropped ', col)

  return df

# Function to handle VIF scores; only pass in features; not the label
def vif(df):
  import pandas as pd
  from sklearn.linear_model import LinearRegression
  
  # initialize dictionaries
  vif_dict, tolerance_dict = {}, {}

  # form input data for each exogenous variable
  for col in df.drop(columns=['const']):
    y = df[col]
    X = df.drop(columns=[col])
    
    # extract r-squared from the fit
    r_squared = LinearRegression().fit(X, y).score(X, y)

    # calculate VIF
    if r_squared < 1: # Prevent division by zero runtime error
      vif = 1/(1 - r_squared) 
    else:
      vif = 100
    vif_dict[col] = vif

    # calculate tolerance
    tolerance = 1 - r_squared
    tolerance_dict[col] = tolerance

    # generate the DataFrame to return
    df_output = pd.DataFrame({'VIF': vif_dict, 'Tolerance': tolerance_dict})

  return df_output.sort_values(by=['VIF'], ascending=False)

In [2]:
!pip install pandas
!pip install sodapy
import json
import pandas as pd
from sodapy import Socrata

# Authenticated client (needed for non-public datasets): This allowed us to avoid API limits
client = Socrata("opendata.utah.gov",
                 "b8AQz7fuwpqdLHiNzouIy8BOl",
                 username="alicialane01@gmail.com",
                 password="d00A@EJ3Ab^ac#J")

# Limit 300,000 to make sure we can get all 252,500 records (this will need to be adjusted if more data is added)
results = client.get("herb-zqda", limit=300000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_dict(results)

In [3]:
#Use this to reset df values/columns without having to rerun previous block of code
collision_df = results_df.copy() 
collision_df.drop(columns = ['crash_id'], inplace = True)
collision_df.shape

(252500, 28)

In [4]:
#convert columns to correct data types since they all came back as object dtype from the API. Data Types determined by data dictionary and model requirements
import numpy as np

collision_df = collision_df.astype({
    'crash_datetime': np.datetime64,
    'milepoint': np.number,
    'lat_utm_y': np.number,
    'long_utm_x': np.number,
    'crash_severity_id': np.number,
    'work_zone_related': np.number,
    'pedestrian_involved': np.number,
    'bicyclist_involved': np.number,
    'motorcycle_involved': np.number,
    'improper_restraint': np.number,
    'unrestrained': np.number,
    'dui': np.number,
    'intersection_related': np.number,
    'wild_animal_related': np.number,
    'domestic_animal_related': np.number,
    'overturn_rollover': np.number,
    'commercial_motor_veh_involved': np.number,
    'teenage_driver_involved': np.number,
    'older_driver_involved': np.number,
    'night_dark_condition': np.number,
    'single_vehicle': np.number,
    'distracted_driving': np.number,
    'drowsy_driving': np.number,
    'roadway_departure': np.number 
})
collision_df.dtypes

crash_datetime                   datetime64[ns]
route                                    object
milepoint                               float64
lat_utm_y                               float64
long_utm_x                              float64
main_road_name                           object
city                                     object
county_name                              object
crash_severity_id                       float64
work_zone_related                       float64
pedestrian_involved                     float64
bicyclist_involved                      float64
motorcycle_involved                     float64
improper_restraint                      float64
unrestrained                            float64
dui                                     float64
intersection_related                    float64
wild_animal_related                     float64
domestic_animal_related                 float64
overturn_rollover                       float64
commercial_motor_veh_involved           

In [5]:
# Use to bin variables (maybe exclude datetime, numeric, boolean, and label columns). City, County Code, and Route are columns that have limited possible values so they could be binned. 
# We didn't include street names as we believe route covers that feature well enough and is better for models.
# We chose .05 percent for the threshold for binning variables since most of the value counts were small. 
# We didn't want to narrow the variables down to only a few values but we also wanted to bin some of the smaller values together.

categorical_df = collision_df[[
  'city',
  'county_name',
  'route'
  ]]

collision_binned = bin_variables(collision_df, categorical_df)
collision_binned.head()

,crash_datetime,route,milepoint,lat_utm_y,long_utm_x,main_road_name,city,county_name,crash_severity_id,work_zone_related,...,domestic_animal_related,overturn_rollover,commercial_motor_veh_involved,teenage_driver_involved,older_driver_involved,night_dark_condition,single_vehicle,distracted_driving,drowsy_driving,roadway_departure
0,2019-02-08 10:56:00,Other,4.560,4513124.085,422620.0512,900 WEST,SALT LAKE CITY,SALT LAKE,2.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2019-12-24 12:48:00,Other,2.702,4483700.743,427458.6694,1215 E HIGHLAND DR,Other,SALT LAKE,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-12-28 20:42:00,Other,0.351,4486609.366,426779.0152,1022 E DRAPER PKY,Other,SALT LAKE,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,2019-12-07 00:46:00,Other,0.438,4484350.913,426397.6578,13463 S FORT ST,Other,SALT LAKE,3.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
4,2019-12-18 17:29:00,Other,0.796,4485217.604,427714.0392,13043 S 1300 E,Other,SALT LAKE,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [6]:
# set main df to new binned df
collision_df = collision_binned.copy()

#handle missing data (there are no columns with 50% or more missing data so we kept all columns)
handle_missing(collision_df, 0.5)

#drop rows containing any missing data since the missing values in total make up less than 3% of the data
collision_df.dropna(axis = 0, how = 'any', inplace= True)
collision_df.shape

(244248, 28)

In [7]:
# convert crash_datetime to days since Jan 1 2016 (since this dataset starts in 2016) 
from datetime import datetime as dt
collision_df['days_since_Jan1'] = (dt.strptime("2023-1-1", '%Y-%m-%d').date() - (pd.to_datetime(collision_df['crash_datetime']).dt.date)).dt.days
collision_df['days_since_Jan1'] = collision_df['days_since_Jan1'].astype(int)

In [10]:
import numpy as np
import pandas.util.testing as tm
import statsmodels.api as sm

# drop columns that won't work in the model:
  # crash_datetime was converted to day_since_Jan1
  # main_road_name is a messing string that would likely need text analytics to use and is partially accounted for in route
model_columns = collision_df.drop(columns=['crash_datetime', 'main_road_name'])
model_columns.dtypes

# Generate dummy variables
for col in model_columns:
  if not pd.api.types.is_numeric_dtype(model_columns[col]):
    model_columns = pd.get_dummies(model_columns, columns=[col], prefix=col, drop_first=True)

# Rename columns to remove spaces created in Dummy coding. Also removed underscores and unified casing so the columns names would be good in the web app.
for (key, value) in model_columns.iteritems() :
    model_columns.rename(columns={key: key.replace('_', ' ').title().replace(' ', '')}, inplace=True)

#print out columns we will use to determine best features and algorithms
model_columns

,Milepoint,LatUtmY,LongUtmX,CrashSeverityId,WorkZoneRelated,PedestrianInvolved,BicyclistInvolved,MotorcycleInvolved,ImproperRestraint,Unrestrained,...,DaysSinceJan1,Route89,RouteOther,CityOther,CitySaltLakeCity,CityWestValleyCity,CountyNameOther,CountyNameSaltLake,CountyNameUtah,CountyNameWeber
0,4.560,4513124.085,422620.0512,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1423,0,1,0,1,0,0,1,0,0
1,2.702,4483700.743,427458.6694,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1104,0,1,1,0,0,0,1,0,0
2,0.351,4486609.366,426779.0152,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1100,0,1,1,0,0,0,1,0,0
3,0.438,4484350.913,426397.6578,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1121,0,1,1,0,0,0,1,0,0
4,0.796,4485217.604,427714.0392,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1110,0,1,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252495,0.100,4484305.829,412209.1538,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1137,0,1,1,0,0,0,1,0,0
252496,1.464,4486291.662,427726.2553,2.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1167,0,1,1,0,0,0,1,0,0
252497,0.998,4483292.486,425009.0526,3.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1164,0,1,1,0,0,0,1,0,0
252498,0.100,4486037.269,428211.6975,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1152,0,1,1,0,0,0,1,0,0


In [16]:
# Determine best combination of features (reran in this same block of code and added columns to the drop() that had p values > .05)
label = 'CrashSeverityId'
model_features = model_columns.drop(axis=1, columns=[
  label, 
  'DaysSinceJan1',
  'Route89',
  'DomesticAnimalRelated',
  'WorkZoneRelated',
  'CountyNameSaltLake',
  'RouteOther', 
  'SingleVehicle', 
  'Milepoint',
  'NightDarkCondition',
  'LatUtmY',
  'RoadwayDeparture'                               
])

# model_features.dtypes
y = model_columns[label]
X = model_features.assign(const=1)

ols_model = sm.OLS(y, X)
ols_model = ols_model.fit()
print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:        CrashSeverityId   R-squared:                       0.185
Model:                            OLS   Adj. R-squared:                  0.185
Method:                 Least Squares   F-statistic:                     2638.
Date:                Thu, 07 Apr 2022   Prob (F-statistic):               0.00
Time:                        21:30:37   Log-Likelihood:            -2.5744e+05
No. Observations:              244248   AIC:                         5.149e+05
Df Residuals:                  244226   BIC:                         5.151e+05
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
LongUtmX            

In [19]:
#calculate VIF scores to see if any need to be dropped (threshold: VIF > 3)
vif(X)

,VIF,Tolerance
CityOther,2.703286,0.369920
CitySaltLakeCity,2.037887,0.490704
CityWestValleyCity,1.892000,0.528541
CountyNameOther,1.570194,0.636864
CountyNameUtah,1.202149,0.831844
WildAnimalRelated,1.176874,0.849708
OverturnRollover,1.117580,0.894791
LongUtmX,1.102995,0.906622
CountyNameWeber,1.100214,0.908914
IntersectionRelated,1.081468,0.924669


In [20]:
# These are defined up here so they didn't get completely wiped of data if the following block was interrupted and had to be restarted
fit = {}    # Use this to store each of the fit metrics
models = {} # Use this to store each of the models

In [31]:
# Determine best classification algorithm (chosen because the label is categorical)

# set parameters used in scoring each model
df = model_features.copy()
label = 'CrashSeverityId'
k=5
r=2
repeat=True

#import needed libraries/tools
import sklearn.linear_model as lm, pandas as pd, sklearn.ensemble as se, numpy as np
from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
from numpy import mean, std
from sklearn import svm
from sklearn import gaussian_process
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn import svm
from sklearn.naive_bayes import CategoricalNB
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier

if repeat:
  cv = RepeatedKFold(n_splits=k, n_repeats=r, random_state=12345)
else:
  cv = KFold(n_splits=k, random_state=12345, shuffle=True)

# Create the model objects
model_log = lm.LogisticRegression(max_iter=100)
model_logcv = lm.RidgeClassifier()
  # model_sgd = lm.SGDClassifier(max_iter=1000, tol=1e-3) - This model takes too long which wouln't be vary compatible with future automated models
model_pa = lm.PassiveAggressiveClassifier(max_iter=1000, random_state=12345, tol=1e-3)
model_per = lm.Perceptron(fit_intercept=False, max_iter=10, tol=None, shuffle=False)
model_knn = KNeighborsClassifier(n_neighbors=3)
  # model_svm = svm.SVC(decision_function_shape='ovo') - This model takes too long which wouln't be vary compatible with future automated models
model_nb = CategoricalNB()
  # model_bag = se.BaggingClassifier(KNeighborsClassifier(), max_samples=0.5, max_features=0.5) - this model skipped (explanation above)
model_ada = se.AdaBoostClassifier(n_estimators=100, random_state=12345)
model_ext = se.ExtraTreesClassifier(n_estimators=100, random_state=12345)
model_rf = se.RandomForestClassifier(n_estimators=10)
model_hgb = se.HistGradientBoostingClassifier(max_iter=100)
model_vot = se.VotingClassifier(estimators=[('lr', model_log), ('rf', model_ext), ('gnb', model_hgb)], voting='hard')
model_gb = se.GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
estimators = [('ridge', lm.RidgeCV()), ('lasso', lm.LassoCV(random_state=12345)), ('knr', KNeighborsRegressor(n_neighbors=20, metric='euclidean'))]
final_estimator = se.GradientBoostingRegressor(n_estimators=25, subsample=0.5, min_samples_leaf=25, max_features=1, random_state=12345)
model_st = se.StackingRegressor(estimators=estimators, final_estimator=final_estimator)
model_xgb = XGBClassifier()
model_nn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=12345)

# Fit a cross-validated R squared score and add it to the dict
fit['Logistic'] = mean(cross_val_score(model_log, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
fit['Ridge'] = mean(cross_val_score(model_logcv, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  # fit['SGD'] = mean(cross_val_score(model_sgd, X, y, scoring='accuracy', cv=cv, n_jobs=-1)) - this model skipped (explanation above)
fit['PassiveAggressive'] = mean(cross_val_score(model_pa, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
fit['Perceptron'] = mean(cross_val_score(model_per, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
fit['KNN'] = mean(cross_val_score(model_knn, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  # fit['SVM'] = mean(cross_val_score(model_svm, X, y, scoring='accuracy', cv=cv, n_jobs=-1)) - this model skipped (explanation above)
fit['NaiveBayes'] = mean(cross_val_score(model_nb, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  # fit['Bagging'] = mean(cross_val_score(model_bag, X, y, scoring='accuracy', cv=cv, n_jobs=-1)) - this model skipped (explanation above)
fit['AdaBoost'] = mean(cross_val_score(model_ada, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
fit['ExtraTrees'] = mean(cross_val_score(model_ext, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
fit['RandomForest'] = mean(cross_val_score(model_rf, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
fit['HistGradient'] = mean(cross_val_score(model_hgb, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
fit['Voting'] = mean(cross_val_score(model_vot, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
fit['GradBoost'] = mean(cross_val_score(model_gb, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
fit['XGBoost'] = mean(cross_val_score(model_xgb, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
fit['NeuralN'] = mean(cross_val_score(model_nn, X, y, scoring='accuracy', cv=cv, n_jobs=-1))

# Add the model to another dictionary; make sure the keys have the same names as the list above
models['Logistic'] = model_log
models['Ridge'] = model_logcv
  # models['SGD'] = model_sgd - this model skipped (explanation above)
models['PassiveAggressive'] = model_pa
models['Perceptron'] = model_per
models['KNN'] = model_knn
  # models['SVM'] = model_svm - this model skipped (explanation above)
models['NaiveBayes'] = model_nb
  # models['Bagging'] = model_bag - this model skipped (explanation above)
models['AdaBoost'] = model_ada
models['ExtraTrees'] = model_ext
models['RandomForest'] = model_rf
models['HistGradient'] = model_hgb
models['Voting'] = model_vot
models['GradBoost'] = model_gb
models['XGBoost'] = model_xgb
models['NeuralN'] = model_nn

# Add the fit dictionary to a new DataFrame, sort, to determine best performing model
df_fit = pd.DataFrame({'Accuracy':fit})
df_fit.sort_values(by=['Accuracy'], ascending=False, inplace=True)
print(df_fit)

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


                   Accuracy
XGBoost            0.720333
Ridge              0.720051
AdaBoost           0.719938
HistGradient       0.719658
Voting             0.712915
Logistic           0.706397
Perceptron         0.706397
GradBoost          0.703394
KNN                0.654026
PassiveAggressive  0.653076
RandomForest       0.606535
ExtraTrees         0.593000
NeuralN            0.095121
NaiveBayes              NaN


In [35]:
from sklearn.model_selection import train_test_split # Import train_test_split function

# convert X and y to arrays because XGBoost model runs better with an array versus a DataFrame.
y = model_columns[label].values
X = model_features.values

#split data for better training and scoring
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# fit the best model 
model = model_xgb.fit(X_train,y_train)
model_xgb.score(X,y) # Accuracy about 72%

predictions = model_xgb.predict(X_test)
output = pd.DataFrame({'Actuals':y_test,'Predictions':predictions});
output #used to make sure the predictions are outputting correctly

,Actuals,Predictions
0,1.0,1.0
1,1.0,1.0
2,2.0,1.0
3,3.0,1.0
4,1.0,1.0
...,...,...
73270,2.0,1.0
73271,3.0,1.0
73272,1.0,1.0
73273,1.0,1.0


In [37]:
# determine number of columns for onnx file settings
X.shape

(244248, 21)

In [38]:
#install tools needed to convert XGBoost model to onnx file
!pip install skl2onnx
!pip install onnxmltools
!pip install onnxruntime

     |████████████████████████████████| 276 kB 14.6 MB/s 
     |████████████████████████████████| 78 kB 6.2 MB/s 
     |████████████████████████████████| 12.8 MB 54.6 MB/s 
     |████████████████████████████████| 300 kB 13.8 MB/s 
     |████████████████████████████████| 5.2 MB 14.1 MB/s 


In [39]:
import skl2onnx as sx
from skl2onnx.common.data_types import FloatTensorType
# from pyquickhelper.helpgen.graphviz_helper import plot_graphviz
# from mlprodict.onnxrt import OnnxInference
# import numpy
import onnxruntime as rt
from sklearn.datasets import load_iris, load_diabetes, make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier, XGBRegressor, DMatrix, train as train_xgb
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn, to_onnx, update_registered_converter
from skl2onnx.common.shape_calculator import (
    calculate_linear_classifier_output_shapes,
    calculate_linear_regressor_output_shapes)
from onnxmltools.convert.xgboost.operator_converters.XGBoost import (
    convert_xgboost)
from onnxmltools.convert import convert_xgboost as convert_xgboost_booster

update_registered_converter(
  XGBClassifier, 'XGBoostXGBClassifier',
  calculate_linear_classifier_output_shapes, convert_xgboost,
  options={'nocl': [True, False], 'zipmap': [True, False, 'columns']})

# initial_type = [('input', FloatTensorType([None, 24]))]
# onnx = sx.convert_sklearn(model, initial_types=initial_type)
# with open("XGBoostClassifierModel.onnx", "wb") as f:
#     f.write(onnx.SerializeToString())

onnx = convert_sklearn(
    model, 'XGBoostClassifierModel',
    [('float_input', FloatTensorType([None, 21]))],
    target_opset={'': 12, 'ai.onnx.ml': 2})

# And save.
with open("XGBoostClassifierModel.onnx", "wb") as f:
    f.write(onnx.SerializeToString())